In [1]:


from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import seaborn as sns
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display
import sys
from scipy.stats import gaussian_kde
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import MaxNLocator
import AnubisSuperScript as ass
from functools import partial
import importlib
importlib.reload(ass)

ERROR:root:Cannot activate multiple GUI eventloops


<module 'AnubisSuperScript' from 'c:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\Scripts\\AnubisSuperScript.py'>

In [2]:

mapping = {
    0: {
        (0, 31): ('rpc0', 'eta'),
        (32, 63): ('rpc0', 'phi1'),
        (64, 95): ('rpc0', 'phi2'),
        (96, 127): ('rpc1', 'eta'),
    },
    1: {
        (0, 31): ('rpc1', 'phi1'),
        (32, 63): ('rpc1', 'phi2'),
        (64, 95): ('rpc2', 'eta'),
        (96, 127): ('rpc2', 'phi1'),
    },
    2: {
        (0, 31): ('rpc2', 'phi2'),
        (32, 63): ('rpc3', 'eta'),
        (64, 95): ('rpc3', 'phi1'),
        (96, 127): ('rpc3', 'phi2'),
    },
    3: {
        (0, 31): ('rpc4', 'eta'),
        (32, 63): ('rpc4', 'phi1'),
        (64, 95): ('rpc4', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

strips_on_steroid = {
    'rpc0': {'phi': [0]},
    'rpc1': {'phi': [0]},
    'rpc2': {'phi': [0]},
    'rpc3': {'phi': [0]},
    'rpc4': {'eta': [31,0], 'phi': [0]},
    'rpc5': {'eta': [31,0], 'phi': [0]},
}


In [3]:
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\60sRun_24_3_4.h5')
df = ass.remake_data(thisData, mapping)

In [4]:
cluster_df = ass.calculate_cluster_metrics_better(df, 5)

In [5]:
remove_noisy_strips_with_map = partial(ass.remove_noisy_strips, noisy_strips=strips_on_steroid)
anti_steriod_df = cluster_df[cluster_df.apply(lambda row: remove_noisy_strips_with_map(row), axis=1)]

In [6]:
fully_massaged_df = anti_steriod_df[anti_steriod_df['size'] <= 5]

In [ ]:
show(anti_steriod_df)

In [25]:
show(fully_massaged_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [11]:
show(cluster_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [51]:
pdf_path1 = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\Olegs Groceries\\rpc_cluster_analysis_all.pdf'
pdf_path2 = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\Olegs Groceries\\rpc_cluster_analysis_anti_steriod.pdf'
# ass.plot_rpc_histograms(cluster_df, pdf_path1)
ass.plot_rpc_histograms(anti_steriod_df, pdf_path2)

In [8]:
pdf_path3 = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\Olegs Groceries\\rpc_cluster_analysis_by_size.pdf'
ass.plot_rpc_histograms_by_cluster_size(anti_steriod_df, pdf_path3)

c:\Users\Peter\OneDrive - University of Cambridge\Desktop\Project Excel Work\PartIIIRPC\ProAnubis_CERN\Scripts\AnubisSuperScript.py:466: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  midpoints_series = pd.Series(all_cluster_midpoints)
c:\Users\Peter\OneDrive - University of Cambridge\Desktop\Project Excel Work\PartIIIRPC\ProAnubis_CERN\Scripts\AnubisSuperScript.py:466: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  midpoints_series = pd.Series(all_cluster_midpoints)
c:\Users\Peter\OneDrive - University of Cambridge\Desktop\Project Excel Work\PartIIIRPC\ProAnubis_CERN\Scripts\AnubisSuperScript.py:466: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning

In [9]:
pdf_path4 = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\Olegs Groceries\\rpc_cluster_time_analysis.pdf'
fully_massaged_df_with_time = fully_massaged_df[fully_massaged_df['end_time'] <= 350]
ass.plot_time_differences_and_event_times(fully_massaged_df_with_time, pdf_path4)

In [7]:
fully_massaged_df_with_time = fully_massaged_df[fully_massaged_df['end_time'] <= 350]

In [9]:
show(fully_massaged_df_with_time)

NameError: name 'fully_massaged_df_with_time' is not defined

In [8]:
def calculate_metric_for_combo(combo, rpc_heights):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])

    try:
        coeffs, cov = np.polyfit(locations, heights, 1, cov=True)
        slope, intercept = coeffs
        slope_error, intercept_error = np.sqrt(np.diag(cov))
        predicted = slope * locations + intercept
        residuals = heights - predicted
        RSS = np.sum(residuals ** 2)
    except np.linalg.LinAlgError:
        slope = np.inf
        intercept = np.inf
        slope_error, intercept_error = np.inf, np.inf
        RSS = np.inf
    
    # combined_uncertainty = np.sum(uncertainties)
    return slope, intercept, slope_error, intercept_error,RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (7.94, 12.48),
    ('rpc0', 'phi'): (-2.38, 13.69),
    ('rpc1', 'eta'): (8.36, 12.22),
    ('rpc1', 'phi'): (-3.79, 13.25),
    ('rpc2', 'eta'): (8.84, 12.56),
    ('rpc2', 'phi'): (-4.35, 13.57),
    ('rpc3', 'eta'): (6.86, 12.41),
    ('rpc3', 'phi'): (-4.3, 13.96),
    ('rpc4', 'eta'): (2.7, 12.37),
    ('rpc4', 'phi'): (-7.89, 13.41),
    ('rpc5', 'eta'): (2.82, 13.05),
    ('rpc5', 'phi'): (9.15, 14.14),
}
    paths = []

    adjusted_muon_speed_cm_ns = 28

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.5, 
        'rpc2': 1.0, 
        'rpc3': 61.5, 
        'rpc4': 121.5, 
        'rpc5': 122.0
    }

    for event_number, event_group in df.groupby('event_number'):
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.09375 if direction == 'eta' else 2.8125
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            for n in range(3, 6):
                for combo in combinations(all_clusters, n):
                    metric = calculate_metric_for_combo(combo, rpc_heights)
                    if metric[4] != np.inf:
                        combination_metrics.append(metric)
                        
            # Move filtering logic outside the loop so it's not reset each time
            for combo_metric in combination_metrics:
                combo = combo_metric[-1]
                if len({c['rpc'] for c in combo}) < len(combo):
                    continue

                time_diffs_are_valid = True
                for i in range(len(combo)):
                    for j in range(i + 1, len(combo)):
                        error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                        
                        # Use direct height differences
                        height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                        
                        time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                        expected_time_diff = height_diff / adjusted_muon_speed_cm_ns

                        uncertainty_margin = 5
                        # Use the expected time difference with the error window and uncertainty margin for validation
                        if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                            time_diffs_are_valid = False
                            break
                    if not time_diffs_are_valid:
                        break

                if time_diffs_are_valid:
                    valid_combinations.append(combo_metric)

            for valid_combination in valid_combinations:
                paths.append({
                    'Event Number': event_number,
                    'Direction': direction,
                    'Slope': valid_combination[0],
                    'Intercept': valid_combination[1],
                    'Slope_error': valid_combination[2],
                    'Intercept_error': valid_combination[3],
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                    'RSS': valid_combination[4],
                })

    path_df = pd.DataFrame(paths)
    return path_df

In [9]:
path_df = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df_with_time)

C:\Users\Peter\AppData\Local\Temp\ipykernel_37660\1504677970.py:6: RankWarning: Polyfit may be poorly conditioned
  coeffs, cov = np.polyfit(locations, heights, 1, cov=True)


In [134]:
show(path_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [10]:
def filter_paths(df):
    excluded_triplet = {'rpc0', 'rpc1', 'rpc2'}

    rows_to_drop = []

    for index, row in df.iterrows():
        used_rpcs = set(c[0] for c in row['Used Coordinates'])

        if used_rpcs.issubset(excluded_triplet):
            rows_to_drop.append(index)  
        elif row['RSS'] > 6000:
            rows_to_drop.append(index) 
    
    # Drop the rows from the DataFrame
    filtered_df = df.drop(rows_to_drop).reset_index(drop=True)
    
    return filtered_df

In [11]:
filtered_dataframe = filter_paths(path_df)

# show(filtered_dataframe)

In [16]:
filtered_df = filtered_dataframe.copy()

def filter_paths():
    global filtered_df
    excluded_triplet = {'rpc0', 'rpc1', 'rpc2'}

    rows_to_drop = []

    for index, row in filtered_df.iterrows():
        used_rpcs = set(c[0] for c in row['Used Coordinates'])

        if used_rpcs.issubset(excluded_triplet):
            rows_to_drop.append(index)
        elif row['RSS'] > 600:
            rows_to_drop.append(index)

    filtered_df = filtered_df.drop(rows_to_drop).reset_index(drop=True)
    update_plot(0)

# GUI setup
root = tk.Tk()
root.title("RPC Event Viewer")
root.geometry("800x600")

frame = tk.Frame(root)
frame.pack(fill=tk.BOTH, expand=True)

current_index = tk.IntVar(value=0)
current_direction = tk.StringVar(value='eta')
selected_hits = tk.IntVar(value=3)

def plot_event(index):
    global filtered_df
    event = filtered_df.iloc[index]
    if event['Direction'] != current_direction.get() or len(event['Used Coordinates']) != selected_hits.get():
        return None
    fig, ax = plt.subplots(figsize = (20, 12))
    x_limits = (-99, 0) if current_direction.get() == 'eta' else (0, 180)
    
    location_scaling = 3.09375 if current_direction.get() == 'eta' else 2.8125

    rpc_heights = {'rpc0': 0, 'rpc1': 0.5, 'rpc2': 1, 'rpc3': 61.5, 'rpc4': 121.5, 'rpc5': 122}
    
    for rpc, height in rpc_heights.items():
        ax.hlines(y=height, xmin=x_limits[0], xmax=x_limits[1], color='gray', linestyle='--')
    
    for coord in event['Used Coordinates']:
        rpc, location, _ = coord
        scaled_location = location * location_scaling
        if x_limits[0] <= scaled_location <= x_limits[1]:
            ax.plot(scaled_location, rpc_heights[rpc], 'ro')
            
    if current_direction.get() == 'eta':
        x_vals = np.linspace(-99, 0, 500)  
    else: 
        x_vals = np.linspace(0, 180, 500) 

    y_vals = event['Slope'] * x_vals + event['Intercept']
    ax.plot(x_vals, y_vals, 'b-')  
    
    ax.set_xlim(x_limits)
    ax.set_ylim(-1, max(rpc_heights.values()) + 1)
    ax.set_xlabel('Location')
    ax.set_ylabel('Height')
    ax.set_title(f'index {index} (Number: {event["Event Number"]}) Direction: {event["Direction"]}')
    
    return fig


def update_plot(index):
    global current_index
    if index < 0 or index >= len(filtered_df):
        return
    current_index.set(index)
    fig = plot_event(index)
    if fig is None: 
        return
    canvas = FigureCanvasTkAgg(fig, master=frame)
    widget = canvas.get_tk_widget()
    widget.grid(row=1, column=0, columnspan=5, sticky='nsew')
    canvas.draw()
    
def filter_and_update():
    global filtered_df
    hits_filter = selected_hits.get()
    filtered_df = filtered_df[filtered_df['Used Coordinates'].apply(len) == hits_filter]
    update_plot(0)

def navigate(direction):
    global filtered_df
    new_index = current_index.get() + direction
    while 0 <= new_index < len(filtered_df):
        if filtered_df.iloc[new_index]['Direction'] == current_direction.get():
            update_plot(new_index)
            break
        new_index += direction

    current_index.set(new_index)

def go_to_index():
    index = int(entry_index.get())
    update_plot(index)
    
current_direction = tk.StringVar(value='eta')


def toggle_direction():
    global filtered_df
    current_direction.set('phi' if current_direction.get() == 'eta' else 'eta')
    new_direction_events = filtered_df[filtered_df['Direction'] == current_direction.get()]
    if not new_direction_events.empty:
        first_relevant_index = new_direction_events.index[0]
        current_index.set(first_relevant_index)
        update_plot(first_relevant_index)
    else:
        print("No events found in the new direction.")
        
label_hits = tk.Label(frame, text="Select number of hits:")
label_hits.grid(row=2, column=0)

combo_hits = ttk.Combobox(frame, textvariable=selected_hits)
combo_hits['values'] = (3, 4, 5, 6) 
combo_hits.grid(row=2, column=1)

# Button to apply the hits filter
button_apply_hits = tk.Button(frame, text="Apply Filter", command=filter_and_update)
button_apply_hits.grid(row=2, column=2)

button_direction = tk.Button(frame, text="Toggle Direction (eta/phi)", command=toggle_direction)
button_direction.grid(row=0, column=4, sticky='ew')

button_prev = tk.Button(frame, text="<< Previous", command=lambda: navigate(-1))
button_prev.grid(row=0, column=0)

button_next = tk.Button(frame, text="Next >>", command=lambda: navigate(1))
button_next.grid(row=0, column=3)

entry_index = tk.Entry(frame)
entry_index.grid(row=0, column=1)

button_go = tk.Button(frame, text="Go to Index", command=go_to_index)
button_go.grid(row=0, column=2)

update_plot(0)

root.mainloop()

C:\Users\Peter\AppData\Local\Temp\ipykernel_33260\3662085476.py:37: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize = (20, 12))


In [29]:
eta_df = filtered_dataframe[filtered_dataframe['Direction'] == 'eta']
angles = np.degrees(np.arctan(eta_df['Slope']))

plt.figure(figsize=(10, 6))
plt.hist(angles, bins=200, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees)')
plt.ylabel('Occurance')
plt.title('Cross_Chamber_Angular_Distribution')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [15]:
phi_df = filtered_dataframe[filtered_dataframe['Direction'] == 'phi']
angles = np.degrees(np.arctan(phi_df['Slope']))

plt.figure(figsize=(10, 6))
plt.hist(angles, bins=200, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees)')
plt.ylabel('Occurance')
plt.title('Cross_Chamber_Angular_Distribution_phi')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [19]:
show(filtered_dataframe)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI
